In [2]:
import numpy as np
import torch

data, y = torch.load('data/house_price_v2.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

np.random.seed(42)

In [ ]:
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor,RandomForestRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier,VotingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer,mean_squared_error

std = StandardScaler()
bagging_svm = BaggingRegressor(base_estimator=SVR())
bagging_svm.set_params(
    **{'base_estimator__C': 59.1069861908854, 'base_estimator__gamma': 8.693959087324277e-05,
       'n_estimators': 96}
)
adaboost = AdaBoostRegressor(base_estimator=SVR())
adaboost_tree = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
adaboost_tree.set_params(**{'learning_rate': 1.0, 'base_estimator__max_depth': 99, 'base_estimator__min_samples_split': 2, 'base_estimator__min_samples_leaf': 7, 'base_estimator__max_features': 71, 'base_estimator__max_leaf_nodes': 1196, 'n_estimators': 999}
                         )
adaboost.set_params(
    **{'base_estimator__C': 10.879585646155862, 'base_estimator__gamma': 0.000506547522932075, 'learning_rate': 0.31729633401624546, 'n_estimators': 295}
)
gdbt = GradientBoostingRegressor()
gdbt.set_params(
    **{'n_estimators': 286, 'learning_rate': 0.1, 'max_depth': 99, 'min_samples_split': 839, 'min_samples_leaf': 1, 'max_features': 223}
)
knn = KNeighborsRegressor()
knn.set_params(**{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
               )
rf = RandomForestRegressor()
rf.set_params(
    **{'n_estimators': 937, 'max_depth': 55, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 64}

)

svc = SVR()
svc.set_params(**{'C': 11.169480627782885, 'gamma': 0.00013315071186570328}
               )
tree = DecisionTreeRegressor()
tree.set_params(
    **{'max_depth': 377, 'min_samples_split': 2, 'min_samples_leaf': 19, 'max_features': 158}
)

extra_tree=ExtraTreesRegressor()
extra_tree.set_params(**{'n_estimators': 204, 'max_depth': 99, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 223}
)

voting_model = VotingRegressor([ ('ada', adaboost), ('gdbt', gdbt),
                                 ('knn', knn), ('svc', svc), ('tree', tree),
                                 ('bagging_svm', bagging_svm),('adaboost_tree',adaboost_tree) ,
                                  ('rf',rf),('extra_tree',extra_tree)], n_jobs=-1,
                                verbose=1)

cross_val_score(voting_model,train_data,y,cv=10,n_jobs=-1,scoring=make_scorer(mean_squared_error,greater_is_better=False),verbose=2).mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


In [43]:
import pandas as pd

best_model=voting_model
y_test = best_model.predict(test_data.values)
result = pd.DataFrame(y_test, index=test_data.index, columns=['SalePrice'], )
result.to_csv('voting.csv')

d:\project\python\.venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
